 # MTH8408 : Méthodes d'optimisation et contrôle optimal
 ## Laboratoire 1: Outils pour l'algèbre linéaire et l'optimisation
Tangi Migot

In [4]:
using Pkg; 
Pkg.activate("jump")

  Activating project at `c:\Users\edwar\Desktop\H23\MTH8408\MTH8408\lab1\jump`


## JuMP: un langage de modélisation

### JuMP ?
JuMP est le language de modélisation pour l'optimisation natif dans Julia. En d'autres termes, on peut modéliser des problèmes d'optimisation en Julia grâce à ce package.

JuMP fait partie de l'organisation [jump-dev](https://jump.dev/) qui propose un ensemble d'outils concernant l'optimisation.

On pourra trouver beaucoup d'informations et d'exemples dans la documentation de JuMP [jump.dev/JuMP.jl/stable/](https://jump.dev/JuMP.jl/stable/)

### Installer JuMP
Comme on l'a fait pour `LinearAlgebra` il faut installer le package `JuMP` via le *package manager*.

In [5]:
Pkg.add("JuMP")

    Updating registry at `C:\Users\edwar\.julia\registries\General.toml`


   Resolving package versions...


    Updating `C:\Users\edwar\Desktop\H23\MTH8408\MTH8408\lab1\jump\Project.toml`
  [b6b21f68] + Ipopt v1.1.0
  [4076af6c] + JuMP v1.6.0
  [b8f27783] + MathOptInterface v1.11.3
    Updating `C:\Users\edwar\Desktop\H23\MTH8408\MTH8408\lab1\jump\Manifest.toml`
  [6e4b80f9] + BenchmarkTools v1.3.2
  [d360d2e6] + ChainRulesCore v1.15.6
  [9e997f8a] + ChangesOfVariables v0.1.4
  [523fee87] + CodecBzip2 v0.7.2
  [944b1d66] + CodecZlib v0.7.0
  [bbf7d656] + CommonSubexpressions v0.3.0
  [34da2185] + Compat v4.5.0
  [864edb3b] + DataStructures v0.18.13
  [163ba53b] + DiffResults v1.1.0
  [b552c78f] + DiffRules v1.12.2
  [ffbed154] + DocStringExtensions v0.9.3
  [f6369f11] + ForwardDiff v0.10.34
  [3587e190] + InverseFunctions v0.1.8
  [b6b21f68] + Ipopt v1.1.0
  [92d709cd] + IrrationalConstants v0.1.1
  [692b3bcd] + JLLWrappers v1.4.1
  [682c06a0] + JSON v0.21.3
  [4076af6c] + JuMP v1.6.0
  [2ab3a3ac] + LogExpFunctions v0.3.19
  [1914dd2f] + MacroTools v0.5.10
  [b8f27783] + MathOptInterface v1

Ensuite, on peut ajouter `JuMP` a notre environement et commencer à l'utiliser.

In [6]:
using JuMP

### Installer un solveur
Dans cet exemple, on va ajouter [`Ipopt`](https://en.wikipedia.org/wiki/IPOPT) qui est un package qui fournit un solveur pour l'optimisation nonlinéaire.

In [7]:
Pkg.add("Ipopt")

   Resolving package versions...


  No Changes to `C:\Users\edwar\Desktop\H23\MTH8408\MTH8408\lab1\jump\Project.toml`
  No Changes 

to `C:\Users\edwar\Desktop\H23\MTH8408\MTH8408\lab1\jump\Manifest.toml`


In [ ]:
#ou alors:
#using Pkg
#Pkg.add("Ipopt")

In [8]:
using Ipopt

In [ ]:
? ipopt

On installe aussi `MathOptInterface` qui permet de communiquer entre le modèle et le solveur.

In [9]:
using MathOptInterface

### Structure d'un modèle JuMP
Tous les modèles écrits avec JuMP ont la même structure:
- un objet `Model`, c'est l'initialisation de notre modèle.
- un objet `Optimizer`, qui spécifie un solveur.
- `Variables`, les variables du problème
- `Objective`, la fonction objectif
- `Constraints`, les contraintes.

#### Le modèle
Tous les problèmes modéliser avec JuMP sont des objets `Model`. Cet objet sera ensuite toujours mentionné lorsque l'on ajoutera des variables, la fonction objectif et des contraintes.

In [3]:
using JuMP
using Ipopt
using MathOptInterface

MonModel = Model()

ArgumentError: ArgumentError: Package JuMP [4076af6c-e467-56ae-b986-b466b2749572] is required but does not seem to be installed:
 - Run `Pkg.instantiate()` to install all recorded dependencies.


Il est également possible d'associer un solveur à ce modèle.

In [ ]:
MonModel = Model(Ipopt.Optimizer) #utilise Ipopt comme solveur

#### Les variables
Une première donnée importante de notre problème d'optimisation est la variable.

In [9]:
@variable(MonModel, 0 <= x <= 10)

LoadError: LoadError: UndefVarError: @variable not defined
in expression starting at c:\Users\edwar\Desktop\H23\MTH8408\MTH8408\lab1\notebook_jump.ipynb:1

In [ ]:
unregister(MonModel, :x) # if you want to remove the variable x from the model

A ce stade, on peut aussi spécifier des contraintes de bornes sur les variables.

In [ ]:
n = 10
l = [1; 2; 3; 4; 5; 6; 7; 8; 9; 10]
u = [10; 11; 12; 13; 14; 15; 16; 17; 18; 19]
@variable(MonModel, l[i] <= x[i=1:n] <= u[i])

ou plus simplement

In [ ]:
@variable(MonModel, xx[i=1:n] >= 0)

#### La fonction objectif
La fonction objectif se définit à l'aide de `@objective` et a trois arguments: le nom du modèle, le type d'optimisation (min ou max), et la fonction.

In [ ]:
c = [2; 3; 4; 5] 
@objective(MonModel, Min, sum(c[i]*x[i] for i in 1:4))

On peut aussi l'écrire sous forme de problème de maximisation.

In [ ]:
@objective(MonModel, Max, sum(c[i]*x[i] for i in 1:4))

Si la fonction objectif est nonlinéaire, on peut être plus précis et utiliser `@NLobjective`.

In [ ]:
@NLobjective(MonModel, Min, exp(x[1]))

#### Les contraintes

In [ ]:
TonModel = Model()
@variable(TonModel, x) #on peut avoir plusieurs types de variables
@variable(TonModel, y)
@constraint(TonModel, 5*x + 3*y <= 5) #une première contrainte

Il peut être utile de nommer une contrainte pour s'y réferer ensuite.

In [ ]:
@constraint(TonModel, un_nom_de_contrainte, 6*x + 4*y >= 5)

On peut aussi ajouter plusieurs contraints simultanément avec @constraints

In [ ]:
@constraints(TonModel, begin
           2x <=  1
            x >= -1
       end)

In [ ]:
TonModel[:un_nom_de_contrainte]

Si la contrainte est nonlinéaire, on peut être plus précis et utiliser `@NLconstraint`.

In [ ]:
@NLconstraint(TonModel, x^2 == 0)

On peut également introduire des contraintes plus génériques à l'aide de fonctions.

In [ ]:
a = [1; -3; 5; 7] 
@variable(TonModel, w[1:4])
@constraint(TonModel, sum(a[i]*w[i] for i in 1:4) <= 3)

On peut utiliser une boucle pour définir plusieurs contraintes.

In [ ]:
@constraint(TonModel, conRef3[i in 1:3], 6*x + 4*y >= 5*i)
#= ou identiquement:
for i in 1:3
    @constraint(yourModel, 6*x + 4*y >= 5*i)
end
=#

#### Exemple d'un code complet: Optimisation linéaire
On résout ici un problème d'optimisation linéaire
$$
\min_{x \in R^n} - x - 2y \ \text{sujet à} \ x + y \leq 1, 0 \leq x,y \leq 1.
$$

In [10]:
model = Model(Ipopt.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Ipopt

On définit les variables.

In [11]:
@variable(model, 0 <= x <= 1)
@variable(model, 0 <= y <= 1)

y

La contrainte $x + y \leq 1$.

In [12]:
@constraint(model, x + y <= 1)

x + y <= 1.0

Enfin, on ajoute la fonction objectif.

In [13]:
@objective(model, Min, -x - 2y)

-x - 2 y

Pour résoudre ce problème, on peut utiliser la fonction `optimize`.

In [14]:
JuMP.optimize!(model)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        2
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        2
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality co

Une fois la résolution terminée, on peut vérifier le status de la résolution.

In [15]:
@show JuMP.has_values(model)
@show JuMP.termination_status(model) == MathOptInterface.LOCALLY_SOLVED
@show JuMP.primal_status(model) == MathOptInterface.FEASIBLE_POINT
@show JuMP.dual_status(model) == MathOptInterface.FEASIBLE_POINT

JuMP.has_values(model) = true
JuMP.termination_status(model) == MathOptInterface.LOCALLY_SOLVED = true
JuMP.primal_status(model) == MathOptInterface.FEASIBLE_POINT = true
JuMP.dual_status(model) == MathOptInterface.FEASIBLE_POINT = true


true

On peut également faire une analyse sur la solution obtenue.

In [16]:
@show JuMP.value(x)              # Old syntax: getvalue(x)
@show JuMP.value(y)              # Old syntax: getvalue(y)
@show JuMP.objective_value(model)       # Old syntax: getobjectivevalue(model)

JuMP.value(x) = -4.860844002533048e-10
JuMP.value(y) = 1.0000000083700527


JuMP.objective_value(model) = -2.000000016254021


-2.000000016254021

#### Exemple d'un code complet: Optimisation nonlinéaire
$$
\min_{x,y} exp(x)+y \text{ s.à } exp(x) + sin(x) <= 0
$$

In [17]:
model = Model(Ipopt.Optimizer)
@variable(model, x)
@variable(model, y)

@NLobjective(model, Min, exp(x)+y)
@NLconstraint(model, exp(x)+sin(x) <=0)

(exp(x) + sin(x)) - 0.0 <= 0

In [18]:
optimize!(model)
println("x = ", value(x), " y = ", value(y))

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        1
Number of nonzeros in Lagrangian Hessian.............:        2

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        1
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        1

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.0000000e+00 1.00e+00 1.20e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00  

In [19]:
termination_status(model)

NORM_LIMIT::TerminationStatusCode = 17

In [20]:
vals = zeros(2)
x_index = JuMP.index(x)
y_index = JuMP.index(y)

MathOptInterface.VariableIndex(2)

In [21]:
d = NLPEvaluator(model)

Nonlinear.Evaluator with available features:
  * :Grad
  * :Jac
  * :JacVec
  * :Hess
  * :HessVec
  * :ExprGraph

In [22]:
MathOptInterface.initialize(d, [:Grad])

In [23]:
MathOptInterface.eval_objective(d, zeros(2))

1.0

In [24]:
∇f = zeros(2)
MathOptInterface.eval_objective_gradient(d, ∇f, zeros(2))
∇f

2-element Vector{Float64}:
 1.0
 1.0

In [25]:
J = zeros(1,2)
MathOptInterface.eval_constraint_jacobian(d, J, zeros(2))
J

1×2 Matrix{Float64}:
 2.0  0.0

#### Plus d'information
Dans ce cours, nous n'allons pas uniquement nous concentrer sur des problèmes linéaires, mais aussi nonlinéaire.
Vous trouverez plus d'informations pour ce cas dans la documentation de jump-dev:

[https://jump.dev/JuMP.jl/stable/tutorials/nonlinear/introduction/](https://jump.dev/JuMP.jl/stable/tutorials/nonlinear/introduction/)

et la documentation de MathOptInterface:

[https://jump.dev/MathOptInterface.jl/stable/](https://jump.dev/MathOptInterface.jl/stable/)

Et ça sera le sujet du projet !

## Exo : f(x₁,x₂) = 100(x₂ - x₁²)² + (1 - x₁)²

In [26]:
model = Model(Ipopt.Optimizer)
@variable(model, x1)
@variable(model, x2)
@NLobjective(model, Min, 100*(x2-x1^2)^2 + (1-x1)^2)

optimize!(model)
println("x1 = ", value(x1), " x2 = ", value(x2))

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.0000000e+00 0.00e+00 2.00e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00  